In [1]:
from google.colab import files
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk,re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.decomposition import LatentDirichletAllocation

from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
import xgboost as xgb

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

uploaded=files.upload()
!pip install "dask[complete]" 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


Saving Dealroom data.csv to Dealroom data.csv
     |████████████████████████████████| 665kB 7.8MB/s 
     |████████████████████████████████| 92kB 6.3MB/s 
     |████████████████████████████████| 102kB 8.3MB/s 
  Created wheel for contextvars: filename=contextvars-2.4-cp36-none-any.whl size=7666 sha256=fc5d4c90c2c655179783e4d1c5bab8079749082a2d071bd644a63b2b7b43cf5b
  Stored in directory: /root/.cache/pip/wheels/a5/7d/68/1ebae2668bda2228686e3c1cf16f2c2384cea6e9334ad5f6de
  Created wheel for locket: filename=locket-0.2.0-cp36-none-any.whl size=4040 sha256=42e903248371887db7ca6dfb193efef9049802781a7142f361c0361f5534df88
  Stored in directory: /root/.cache/pip/wheels/26/1e/e8/4fa236ec931b1a0cdd61578e20d4934d7bf188858723b84698
Successfully built contextvars locket
ERROR: distributed 2.30.0 has requirement cloudpickle>=1.5.0, but you'll have cloudpickle 1.3.0 which is incompatible.
  Found existing installation: distributed 1.25.3
    Uninstalling distributed-1.25.3:
      Successfully unins

In [2]:
import dask.dataframe as dd
df=pd.read_csv('Dealroom data.csv',encoding= 'unicode_escape')
df

,NAME,TAGLINE,TAGS,HQ CITY,LAUNCH DATE,GROWTH STAGE,TYPE
0,63336,Ai-enabled q&a service that answers to various...,mobile,London,"2002, September",late growth stage,NaN
1,@Futsal,Educational courses through the medium of spor...,NaN,Birmingham,2008,early growth stage,NaN
2,#5 Magazine,Multi-platform digital lifestyle magazines abo...,publishing;branding;media;platform;entertainment,London,2007,NaN,NaN
3,03Numbers,Planet Numbers are the leading provider of 03 ...,NaN,NaN,"2008, September",early growth stage,NaN
4,077football News & Media,"077Football - the deep-rooted, hyperlocal foot...",sport;advertising;football;network;game develo...,Cambridge,2009,late growth stage,NaN
...,...,...,...,...,...,...,...
11577,Zyentia,"Biotech company, focuses on protein folding, c...",biotechnology;diabetes;health;mortgages & lend...,Cambridge,2002,NaN,NaN
11578,Zyex,"Zyex providing monofilament, multifilament and...",industrial technologies;aviation & aerospace,Stonehouse,1994,early growth stage,NaN
11579,Zynk Software Limited,Sage Integration Software - Zynk,e-commerce;software;integrations system,Newcastle upon Tyne,"2002, May",early growth stage,NaN
11580,Zytronic,Zytronic are developers and manufacturers of p...,deep tech;industrial technologies;recognition ...,Blaydon-on-Tyne,1999,NaN,NaN


In [3]:
df=df.drop(['TYPE','GROWTH STAGE','HQ CITY'],axis=1)
df.TAGS=df.TAGS.fillna(' ')
df.TAGLINE=df.TAGLINE.fillna(' ')

In [4]:
# Tags Cleaning

w=WordNetLemmatizer()
tags=df.TAGS.copy()
for i in range(0,len(tags)):
  # Remove special chara and replace with space
  tags[i] =re.sub("[^a-zA-Z]", " ",tags[i]) 
  # Tokenize
  tags[i] =nltk.word_tokenize(tags[i])
  # All to lowercase
  tags[i] = [l.lower() for l in tags[i]]
  # Remove stop words
  tags[i]=[l for l in tags[i] if l not in stopwords.words('english')]
  # Lemmatization
  tags[i]=[w.lemmatize(l) for l in tags[i]]
  # Join
  # tags[i]=' '.join(tags[i])

# # Joining ecommerce to one word
for i in range(0,len(tags)):
  for j in range(0,len(tags[i])):
    if(j+1<len(tags[i])):
      if(tags[i][j]=='e' and tags[i][j+1]=='commerce'):
          tags[i][j+1]=tags[i][j]+tags[i][j+1]
          del tags[i][j]


In [5]:
# Tagline Cleaning

w=WordNetLemmatizer()
tagline=df.TAGLINE.copy()
for i in range(0,len(tagline)):
  # Remove special chara and replace with space
  tagline[i] =re.sub("[^a-zA-Z]", " ",tagline[i]) 
  # Tokenize
  tagline[i] =nltk.word_tokenize(tagline[i])
  # All to lowercase
  tagline[i] = [l.lower() for l in tagline[i]]
  # Remove stop words
  tagline[i]=[l for l in tagline[i] if l not in stopwords.words('english')]
  # Lemmatization
  tagline[i]=[w.lemmatize(l) for l in tagline[i]]
  #Join
  # tagline[i]=' '.join(tagline[i])

In [6]:
# Datetime Cleaning
df['LAUNCH DATE']=df['LAUNCH DATE'].apply(lambda x: pd.to_datetime(x,errors='coerce'))
df['LAUNCH DATE']=df['LAUNCH DATE'].dt.year
df['LAUNCH DATE']=df['LAUNCH DATE'].fillna(0)
df['LAUNCH DATE']=df['LAUNCH DATE'].astype('int64')

In [7]:
df=df.drop(['TAGLINE','TAGS'],axis=1)
tags=tags+tagline
df['TAGS'] = pd.Series(tags, index=df.index)

In [11]:
## Finding Keywords

topics=list(df.TAGS)
tag=[]
for i in range(0,len(topics)):
  for j in range(0,len(topics[i])):
     if topics[i][j] != '':
        tag.append(topics[i][j])

vector=CountVectorizer()
tag_1=vector.fit_transform(tag)
sum_words = tag_1.sum(axis=0) 
wordcount = [(word, sum_words[0, idx]) for word, idx in  vector.vocabulary_.items()]
wordcount =sorted(wordcount, key = lambda x: x[1], reverse=True)
wordcount100=[l for l in wordcount if l[1]>150]
wordcount100

[('technology', 3001),
 ('software', 2973),
 ('service', 2552),
 ('management', 2197),
 ('medium', 1771),
 ('data', 1711),
 ('design', 1675),
 ('mobile', 1575),
 ('solution', 1484),
 ('platform', 1367),
 ('online', 1346),
 ('social', 1328),
 ('company', 1323),
 ('business', 1262),
 ('tech', 1123),
 ('uk', 974),
 ('subscription', 889),
 ('finance', 883),
 ('ecommerce', 863),
 ('cloud', 841),
 ('marketing', 823),
 ('digital', 817),
 ('development', 792),
 ('search', 772),
 ('deep', 754),
 ('risk', 753),
 ('advertising', 748),
 ('community', 736),
 ('engine', 717),
 ('compliance', 709),
 ('video', 705),
 ('game', 690),
 ('based', 650),
 ('home', 641),
 ('system', 634),
 ('provider', 632),
 ('agency', 623),
 ('content', 612),
 ('app', 610),
 ('product', 603),
 ('trading', 598),
 ('web', 577),
 ('delivery', 574),
 ('retail', 542),
 ('energy', 532),
 ('saas', 523),
 ('leading', 516),
 ('communication', 510),
 ('analytics', 509),
 ('enterprise', 484),
 ('market', 478),
 ('branding', 472),
 ('

In [12]:
tech_kw=['social','subscription','regtech','adtech','programme','technology','technological','medium','hosting','web','fintech','cleantech','game','website','cloud','app','engine','tech','software','data','mobile','platform','analytics','ecommerce','biotechnology','automated','digital','insurtech','wireless','engineering','telecommunication','gaming','virtual','saas','online','data']
uni_kw=['schools','university','college']
tags=df.TAGS.copy()
for i in range(0,len(tags)):
  if any(item in tags[i] for item in tech_kw):
     tags[i]=1
  elif any(item in tags[i] for item in uni_kw):
    tags[i]=2
  else:
    tags[i]=0

In [14]:
df['TAGSENCOD'] = pd.Series(tags, index=df.index)
df.TAGSENCOD=df.TAGSENCOD.astype('int64')

startups=df[(df['LAUNCH DATE']>1990) & (df['TAGSENCOD']==1)]
df=df.drop(startups.index)
print(startups)

MNC=df[(df['LAUNCH DATE']<=1990) & (df['TAGSENCOD']!=2) ]
df=df.drop(MNC.index)
print(MNC)

University=df[df['TAGSENCOD']==2 ]
df=df.drop(University.index)
print(University)

Unclassified=df.copy

df

                                   NAME  ...  TAGSENCOD
0                                 63336  ...          1
1                               @Futsal  ...          1
2                           #5 Magazine  ...          1
4              077football News & Media  ...          1
5      077football News & Media Limited  ...          1
...                                 ...  ...        ...
11577                           Zyentia  ...          1
11578                              Zyex  ...          1
11579             Zynk Software Limited  ...          1
11580                          Zytronic  ...          1
11581                        ??????:520  ...          1

[7936 rows x 4 columns]
                                   NAME  ...  TAGSENCOD
164         ABN AMRO Commercial Finance  ...          0
188           Acal Enterprise Solutions  ...          1
221                      Achilles Group  ...          1
224                               Acora  ...          0
268                    

,NAME,LAUNCH DATE,TAGS,TAGSENCOD
3,03Numbers,2008,"[planet, number, leading, provider, number, uk...",0
11,100%Open,2009,"[cancer, oncology, delivery, design, risk, man...",0
15,10Associates,2004,"[successful, brand]",0
17,111pix,2006,"[uk, based, independent, production, internati...",0
20,123 Reg,2000,"[local, largest, domain, provider]",0
...,...,...,...,...
11554,Zoggs International,1992,"[branding, global, swimming, product, brand]",0
11557,Zomp,2003,"[provider, consumer, electronics]",0
11559,Zone3,2008,"[sport, wear, sport, wear, accessory]",0
11565,Zoom Abroad,2010,"[education, zoom, abroad, rated, one, friendly...",0
